**Lab 3: Contextual Bandit-Based News Article Recommendation**

Course: Reinforcement Learning Fundamentals

Student Name: Sai Chinmayi Kalapatapu

Roll Number: U20230085

GitHub Branch: chinmayi_U20230085


In [44]:
pip install numpy pandas matplotlib scikit-learn rlcmab-sampler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.4.2 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.4.2 which is incompatible.


# Installing Packages

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

from rlcmab_sampler import sampler

#Load The Datasets

In [9]:
# Load datasets
news_df = pd.read_csv("data/news_articles.csv")
train_users = pd.read_csv("data/train_users.csv")
test_users = pd.read_csv("data/test_users.csv")

print(news_df.head())
print(train_users.head())

                                                link  \
0  https://www.huffpost.com/entry/covid-boosters-...   
1  https://www.huffpost.com/entry/american-airlin...   
2  https://www.huffpost.com/entry/funniest-tweets...   
3  https://www.huffpost.com/entry/funniest-parent...   
4  https://www.huffpost.com/entry/amy-cooper-lose...   

                                            headline   category  \
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  He was subdued by passengers and crew when he ...        Mary Papenfuss   
2  "Until you have a dog y

# Data Analysis

In [10]:
print("Missing values in train_users:")
print(train_users.isnull().sum())

print("Missing values in test_users:")
print(test_users.isnull().sum())

print("Dtypes in train_users:")
print(train_users.dtypes)

print("Unique labels in train_users:")
print(train_users['label'].unique())
print(f"Label distribution:\n{train_users['label'].value_counts()}")

print("Summary statistics for train_users:")
print(train_users.describe())

Missing values in train_users:
user_id                          0
age                            698
income                           0
clicks                           0
purchase_amount                  0
session_duration                 0
content_variety                  0
engagement_score                 0
num_transactions                 0
avg_monthly_spend                0
avg_cart_value                   0
browsing_depth                   0
revisit_rate                     0
scroll_activity                  0
time_on_site                     0
interaction_count                0
preferred_price_range            0
discount_usage_rate              0
wishlist_size                    0
product_views                    0
repeat_purchase_gap (days)       0
churn_risk_score                 0
loyalty_index                    0
screen_brightness                0
battery_percentage               0
cart_abandonment_count           0
browser_version                  0
background_app_count    

In [11]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

train_users_proc = train_users.drop(columns=['user_id'])
test_users_proc = test_users.drop(columns=['user_id'])

X = train_users_proc.drop(columns=['label'])
y = train_users_proc['label']

age_imputer = SimpleImputer(strategy='median')
X['age'] = age_imputer.fit_transform(X[['age']])
test_users_proc['age'] = age_imputer.transform(test_users_proc[['age']])

categorical_cols = ['browser_version', 'region_code']
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X[categorical_cols] = ordinal_encoder.fit_transform(X[categorical_cols])
test_users_proc[categorical_cols] = ordinal_encoder.transform(test_users_proc[categorical_cols])

X['subscriber'] = X['subscriber'].astype(int)
test_users_proc['subscriber'] = test_users_proc['subscriber'].astype(int)

print("Final feature matrix shape (train):", X.shape)
print("Final feature matrix shape (test):", test_users_proc.shape)
print("Missing values in X:", X.isnull().sum().sum())
print("Missing values in test:", test_users_proc.isnull().sum().sum())
print("Unique labels:", y.unique())

Final feature matrix shape (train): (2000, 31)
Final feature matrix shape (test): (2000, 31)
Missing values in X: 0
Missing values in test: 0
Unique labels: ['user_3' 'user_2' 'user_1']


# 5.2 User Classification

Tested out different models for classification but found xgboost to give the highest accuracy value

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [16]:
pip install xgboost

In [18]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_val_enc = label_encoder.transform(y_val)

xgb_model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    objective="multi:softmax",
    num_class=3,
    eval_metric="mlogloss",
    random_state=42
)

xgb_model.fit(X_train, y_train_enc)
xgb_pred = xgb_model.predict(X_val)
xgb_acc = accuracy_score(y_val_enc, xgb_pred)

print(f"XGBoost Validation Accuracy: {xgb_acc:.4f}")

XGBoost Validation Accuracy: 0.9025
